In [1]:
import numpy as np
import pandas as pd

In [2]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [4]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA2b/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'oth'],
      dtype='object')

In [5]:
mineralogy = mineralogy.dropna()

In [6]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [7]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["oth"], axis=1))

In [8]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [9]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
6960,68.584550,0.489185,15.482604,1.096601,3.055998,0.092514,1.918629,2.950502,3.185583,3.143834
6961,68.584242,0.489204,15.482589,1.096791,3.055951,0.092516,1.918842,2.950605,3.185561,3.143699
6962,68.583941,0.489222,15.482575,1.096980,3.055903,0.092518,1.919052,2.950705,3.185539,3.143566
6963,68.583648,0.489239,15.482560,1.097165,3.055855,0.092520,1.919257,2.950803,3.185518,3.143435
6964,68.583363,0.489256,15.482544,1.097347,3.055807,0.092522,1.919458,2.950898,3.185498,3.143307


In [10]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
6960,68.584550,0.489185,15.482604,1.096601,3.055998,0.092514,1.918629,2.950502,3.185583,3.143834
6961,68.584242,0.489204,15.482589,1.096791,3.055951,0.092516,1.918842,2.950605,3.185561,3.143699
6962,68.583941,0.489222,15.482575,1.096980,3.055903,0.092518,1.919052,2.950705,3.185539,3.143566
6963,68.583648,0.489239,15.482560,1.097165,3.055855,0.092520,1.919257,2.950803,3.185518,3.143435
6964,68.583363,0.489256,15.482544,1.097347,3.055807,0.092522,1.919458,2.950898,3.185498,3.143307
...,...,...,...,...,...,...,...,...,...,...
262824,74.272416,0.163680,14.067958,0.581376,1.705941,0.056071,0.400913,1.058031,3.338710,4.354904
262825,74.272592,0.163673,14.067886,0.581395,1.705881,0.056070,0.400898,1.057990,3.338709,4.354906
262826,74.272748,0.163667,14.067820,0.581406,1.705829,0.056069,0.400883,1.057952,3.338709,4.354916
262827,74.272885,0.163661,14.067761,0.581411,1.705786,0.056068,0.400867,1.057917,3.338710,4.354934


In [11]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45000:90000]
mineralogy3 = mineralogy.iloc[90000:135000]
mineralogy4 = mineralogy.iloc[135000:160000]

In [12]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA2b/area2b_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA2b/area2b_2.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy3, "../_CIPW/CIPW/AREA2b/area2b_3.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy4, "../_CIPW/CIPW/AREA2b/area2b_4.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [14]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA2b/area2b_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA2b/area2b_2_results_excel.txt")
df_final3 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA2b/area2b_3_results_excel.txt")
df_final4 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA2b/area2b_4_results_excel.txt")

In [15]:
df_final = pd.concat([df_final1, df_final2, df_final3, df_final4])

In [16]:
df_final

,Q,P,K
SAMPLE,,,
6960,26.985256,38.894960,21.272655
6961,26.985134,38.895351,21.271870
6962,26.985039,38.895696,21.271086
6963,26.984987,38.895991,21.270302
6964,26.984917,38.896334,21.269515
...,...,...,...
262824,34.185909,30.672937,28.558591
262825,34.186180,30.672738,28.558650
262826,34.186470,30.672540,28.558709


## QAPF classification

In [17]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [18]:
df_final["QAPF"].unique()

array(['monzo granite', 'granodiorite'], dtype=object)

In [203]:
df_final.to_excel("../_CIPW/CIPW/AREA2b/qapf.xlsx", index=True)

-------

In [19]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA2b/QAPF_counts.xlsx")

-----

In [20]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area2_b.xlsx", index_col=0, usecols = [0, 1, 2, 3, 4, 29])

In [21]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
13,566277.769165,5.115580e+06,53.0,T,7.0,42.445293,29.966574,23.523890,monzo granite
38,511633.869355,5.052403e+06,53.0,T,7.0,43.370583,25.742728,27.407559,monzo granite
64,535530.362355,5.123776e+06,53.0,T,7.0,39.431854,27.592831,30.132075,monzo granite
82,523597.931141,5.094252e+06,53.0,T,7.0,44.388229,14.692972,36.481345,syeno granite
149,529209.364967,5.079916e+06,53.0,T,7.0,39.287922,26.193204,29.274060,monzo granite
...,...,...,...,...,...,...,...,...,...
4267,531250.244842,5.036709e+06,53.0,T,7.0,15.171756,46.716003,17.795245,quartz monzodiorite\nquartz monzogabbro
4307,561486.936139,5.141199e+06,53.0,T,7.0,17.179220,44.442557,16.281517,granodiorite
4508,563260.439707,5.114646e+06,53.0,T,7.0,4.372070,44.536356,27.149402,quartz monzonite
4601,564967.071345,5.113837e+06,53.0,T,7.0,2.796842,45.321224,21.422379,monzodiorite monzogabbro


In [23]:
QAPF_control.to_excel("../_CIPW/CIPW/AREA2b/QAPF_control_points.xlsx", index=True)